In [1]:
import pysimdeum
import pysimdeum.utils.wastewater_quality as wq
from pysimdeum.core.house import Property
from pysimdeum.core.statistics import Statistics
import pandas as pd
from datetime import datetime

In [9]:
file_path = 'discharge_data_sim_UK.xlsx'
sheet_name='Data'

In [10]:
enduse = 'Wc'
impact = 'high'
date = datetime(2025, 3, 31).date()
start = 0
end = 10

In [ ]:
def pySimdeum_run(houses):
    for i in range(start, end, 1):
        house = pysimdeum.built_house(house_type=houses, duration='5 days', country='UK', simulate_discharge=True, spillover=True)
        #consumption, discharge = house.simulate(date=date, duration='5 days', num_patterns=1, simulate_discharge=True, spillover=True)

        if [event for event in house.discharge.discharge_events.values if event['enduse'] == enduse] and (enduse in house.discharge.discharge.coords['enduse'].values) and (house.discharge.discharge.sum(["user", "dischargetypes"]).sel(enduse=enduse).sum() > 0):
            print("enduse events exist")
            hh_nutrients = wq.rat_hh_discharge_nutrients(house.discharge, enduse, country='UK', time_agg='30min')
            print("nutrients done")
            hh_temp = wq.rat_hh_discharge_temperature(house.discharge, enduse, time_agg='30min')
            print("temp done")
            hh_combined = pd.merge(hh_nutrients, hh_temp[['time', 'enduse', 'discharge_temperature']], on=['time', 'enduse'], how='outer')
            print("combined done")
            enduse_df = hh_combined

            # Ensure all timestamps exist (fill missing times with zero values)
            ref_start, ref_end = hh_combined['time'].min(), hh_combined['time'].max()
            full_time_index = pd.date_range(start=ref_start, end=ref_end, freq='30min')
            if not enduse_df.empty:
                enduse_df = enduse_df.set_index('time').reindex(full_time_index, fill_value=0).reset_index()
                enduse_df = enduse_df.rename(columns={'index': 'time'})
            else:
                # If no data exists for the end-use, create a full zero-filled DataFrame
                enduse_df = pd.DataFrame({'time': full_time_index, 'flow': 0, 'discharge_temperature': 0, 'house_id': house.id, 'enduse': enduse})

        else:
            print("No enduse events found")
            ref_start = pd.Timestamp.now().normalize()
            ref_end = ref_start + pd.Timedelta(days=10)
            full_time_index = pd.date_range(start=ref_start, end=ref_end, freq='30min')
            enduse_df = pd.DataFrame({'time': full_time_index, 'enduse': enduse, 'flow': 0, 'n': 0, 'p': 0, 'cod': 0, 'bod5': 0, 'ss': 0, 'amm': 0, 'discharge_temperature': 0})

        print("filled done")
        # Assign a unique identifier to the house
        enduse_df['house_id'] = house.id
        enduse_df['number'] = i
        enduse_df['scenario'] = impact
        enduse_df['house_type'] = houses

        # Append to excel
        try:
            with pd.ExcelWriter(file_path, mode='a', if_sheet_exists='overlay', engine='openpyxl') as writer:
                existing_df = pd.read_excel(file_path, sheet_name=sheet_name)
                enduse_df = pd.concat([existing_df, enduse_df], ignore_index=True)  # Append new data
                enduse_df.to_excel(writer, sheet_name=sheet_name, index=False)  # Save back
        except FileNotFoundError:
            # If file doesn't exist, create a new one
            with pd.ExcelWriter(file_path, mode='w', engine='openpyxl') as writer:
                enduse_df.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f"house {i} processed")

In [12]:
pySimdeum_run(houses='one_person')

enduse events exist
nutrients done
temp done
combined done
filled done
house 0 processed
enduse events exist
nutrients done
temp done
combined done
filled done
house 1 processed
enduse events exist
nutrients done
temp done
combined done
filled done
house 2 processed
enduse events exist
nutrients done
temp done
combined done
filled done
house 3 processed
enduse events exist
nutrients done
temp done
combined done
filled done
house 4 processed
enduse events exist
nutrients done
temp done
combined done
filled done
house 5 processed
A usage event for  WashingMachine  use has spilled over to the next day. Adjusting spillover times...
Spillover consumption adjustment complete.
enduse events exist
nutrients done
temp done
combined done
filled done
house 6 processed
enduse events exist
nutrients done
temp done
combined done
filled done
house 7 processed
enduse events exist
nutrients done
temp done
combined done
filled done
house 8 processed
enduse events exist
nutrients done
temp done
combined 